# Create chemical clusters based on their similarities
Note: to run this code you need to have RDkit package and environment

In [38]:
import os
import pandas as pd
from rdkit.Chem.Fingerprints import ClusterMols
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import PandasTools
from rdkit.Chem import rdFingerprintGenerator
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdmolops
from rdkit.Chem.Fingerprints import FingerprintMols
import numpy as np
from tqdm import tqdm_notebook as tqdm
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
import pybel
from se_kge.get_url_requests import cid_to_smiles

# Get chemicals mapping file

In [21]:
mapping_file = pd.read_csv(os.path.join(os.pardir, "resources", "mapping","drugbank_pubchem_mapping.tsv"), sep='\t' ,dtype={'PubchemID': str, 'Smiles': str})

In [22]:
mapping_file.head()

,PubchemID,DrugbankID,DrugbankName,Smiles
0,16129704,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CCC(O)=O)NC(=O)[C@H](CC1=CC=CC=C1)NC(=O)[C@H](CC(O)=O)NC(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=O)[C@@H]1CCCN1C(=O)[C@H](N)CC1=CC=CC=C1)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CCC(O)=O)C(=O)N[C@@H](CC1=CC=C(O)C=C1)C(=O)N[C@@H](CC(C)C)C(O)=O
1,5311128,DB00014,Goserelin,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@H](CC1=CNC2=CC=CC=C12)NC(=O)[C@H](CC1=CN=CN1)NC(=O)[C@@H]1CCC(=O)N1)C(=O)N[C@@H](CCCN=C(N)N)C(=O)N1CCC[C@H]1C(=O)NNC(N)=O
2,45267103,DB00027,Gramicidin D,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(=O)N[C@@H](C)C(=O)N[C@H](C(C)C)C(=O)N[C@@H](C(C)C)C(=O)N[C@H](C(C)C)C(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)N[C@H](CC(C)C)C(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)N[C@H](CC(C)C)C(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)N[C@H](CC(C)C)C(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)NCCO
3,16051933,DB00035,Desmopressin,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)[C@H](CC2=CC=C(O)C=C2)NC(=O)CCSSC[C@H](NC(=O)[C@H](CC(N)=O)NC1=O)C(=O)N1CCC[C@H]1C(=O)N[C@@H](CCCNC(N)=N)C(=O)NCC(N)=O
4,25074887,DB00050,Cetrorelix,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H](CC1=CC=C(O)C=C1)NC(=O)[C@H](CO)NC(=O)[C@@H](CC1=CN=CC=C1)NC(=O)[C@@H](CC1=CC=C(Cl)C=C1)NC(=O)[C@@H](CC1=CC2=CC=CC=C2C=C1)NC(C)=O)C(=O)N[C@@H](CCCNC(N)=N)C(=O)N1CCC[C@H]1C(=O)N[C@H](C)C(N)=O


In [23]:
mapping_file = mapping_file.dropna()

In [24]:
full_graph = pybel.from_pickle(os.path.join(os.pardir, "resources", "basic_graphs", "fullgraph_without_sim.pickle"))

In [25]:
chem_list = []
for node in full_graph.nodes():
    if node.namespace != 'pubchem':
        continue
    chem_list.append(node.identifier)

In [26]:
mols_dict = {}
for index, row in mapping_file.iterrows():
    if str(row['PubchemID']) not in chem_list:
        continue
    mols_dict[row['PubchemID']] = Chem.MolFromSmiles(row['Smiles'])

In [40]:
for chem in tqdm(chem_list):
    if chem not in mols_dict.keys():
        smiles = cid_to_smiles(chem)
        if not isinstance(smiles, str):
            smiles = smiles.decode("utf-8")
        if smiles is None:
            continue
        mols_dict[chem] = Chem.MolFromSmiles(smiles)

# Using distance matrix for clustering

# Clustering using Butina

In [42]:
fps_drug = {}
fps = []
drugs = []
for drug, mol in tqdm(mols_dict.items()):
    if mol is None:
        continue
    fp = MACCSkeys.GenMACCSKeys(mol)
    fps.append(fp)
    drugs.append(drug)
    fps_drug[drug] = fp

In [43]:
dists = []
nfps = len(fps)
for i in tqdm(range(1,nfps)):
    sims = DataStructs.BulkTanimotoSimilarity(fps[i],fps[:i])
    dists.extend([1-x for x in sims])

In [44]:
cs = Butina.ClusterData(dists,nfps,0.3,isDistData=True)

In [45]:
df = pd.DataFrame(columns = ['PubchemID', 'Cluster'])

In [46]:
i=1
j=1
clusters = {}
for cluster in cs:
    for drug in cluster:
        df.loc[i] = [drugs[drug-1]] + [j]
        i+=1
    j+=1

In [47]:
df.to_csv(os.path.join(os.pardir, "resources", "mapping","Clustered_chemicals.csv"), index=False)